# base setting

In [1957]:
# Google drive mout
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1958]:
# Library import
import pandas as pd

import numpy as np
from numpy import linalg

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import cm
from matplotlib.ticker import LinearLocator

import seaborn as sns

import math

from scipy import stats

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.graphics.factorplots import interaction_plot

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


import missingno as msno
import urllib
import re
import random

from datetime import datetime


import sympy
# Juypter 노트북에서 수학식의 LaTeX 표현을 위해 필요함
sympy.init_printing(use_latex='mathjax')


# from vega_datasets import data
# from tabulate import tabulate

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# plt.style.use('seaborn')
# plt.style.use('default')
# sns.set(font_scale=1)

# pandas loat format 설정
# pd.options.display.float_format = '{:,.2f}'.format      # 천단위 콤마찍고, 소수점 둘째자리까지 표시
pd.options.display.float_format = None                  # float 포멧 초기화

In [1959]:
# 그래프 한글폰트 개선
# plt.rcParams['font.family'] ='Malgun Gothic'
# plt.rcParams['axes.unicode_minus'] =False
# plt.rc('font', family='NanumGothic')

In [1960]:
# 그래프 한글폰트 개선
# !apt -qq -y install fonts-nanum > /dev/null
 
# fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# font = fm.FontProperties(fname=fontpath, size=10)
# fm._rebuild()
 
# # 그래프에 retina display 적용
# %config InlineBackend.figure_format = 'retina'
 
# # Colab 의 한글 폰트 설정
# plt.rc('font', family='NanumBarunGothic') 

# VGCartz



In [1961]:
VGC_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/vgchartz.csv'
df_raw = pd.read_csv(VGC_path) # read
print(df_raw.shape)            # shape
df_raw.head()                  # head()
# Unnamed: 0 컬럼은 불필요함

(62437, 15)


,Unnamed: 0,Genre,Name,url,Platform,Publisher,VGC_Score,Critic_Score,User_Score,Total_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year
0,0,Action,God of War,https://www.vgchartz.com/game/226282/god-of-war/,Series,Sony Computer Entertainment,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,22nd Mar 05
1,1,Action,Warriors,https://www.vgchartz.com/game/226375/warriors/,Series,KOEI,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,30th Jun 97
2,2,Action,Devil May Cry,https://www.vgchartz.com/game/226043/devil-may...,Series,Capcom,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,16th Oct 01
3,3,Action,God of War (2018),https://www.vgchartz.com/game/230391/god-of-wa...,All,Sony Interactive Entertainment,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,20th Apr 18
4,4,Action,Dynasty Warriors,https://www.vgchartz.com/game/226374/dynasty-w...,Series,KOEI,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,30th Jun 97


## EDA




In [1962]:
df_raw.columns

Index(['Unnamed: 0', 'Genre', 'Name', 'url', 'Platform', 'Publisher',
       'VGC_Score', 'Critic_Score', 'User_Score', 'Total_Sales', 'NA_Sales',
       'PAL_Sales', 'JP_Sales', 'Other_Sales', 'Year'],
      dtype='object')

In [1963]:
# 불필요한 컬럼 제거
df_raw.drop(columns=['Unnamed: 0'], inplace=True)
df_raw.columns

Index(['Genre', 'Name', 'url', 'Platform', 'Publisher', 'VGC_Score',
       'Critic_Score', 'User_Score', 'Total_Sales', 'NA_Sales', 'PAL_Sales',
       'JP_Sales', 'Other_Sales', 'Year'],
      dtype='object')

### Name

In [1964]:
col = df_raw.Name
print(f"""{col.name} 컬럼 정보
- 데이터타입        : {col.dtype}
- shape             : {col.shape}
- 결측치 갯수       : {col.isna().sum()}
- 중복값 갯수       : {col.duplicated().sum()}
- 유니크값 갯수     : {len(col.unique())}
""")

Name 컬럼 정보
- 데이터타입        : object
- shape             : (62437,)
- 결측치 갯수       : 0
- 중복값 갯수       : 22946
- 유니크값 갯수     : 39491



In [1965]:
# 좌우공백제거
df_raw.Name = df_raw.Name.str.strip()

In [1966]:
# 중복값 보기
Name_count = pd.DataFrame(df_raw.Name.value_counts())
filt_p = Name_count.Name != 1
Name_count[filt_p]

,Name
Plants vs. Zombies,19
Monopoly,15
Doom,14
Elite,14
Double Dragon,14
...,...
60 Seconds!,2
Heart of the Woods,2
"Second Novel: Kanojo no Natsu, 15-Bun no Kioku",2
Root Film,2


In [1967]:
# Plants vs. Zombies 이 왜 중복된 데이터가 많은지 확인
df_raw[df_raw.Name == 'Plants vs. Zombies']

,Genre,Name,url,Platform,Publisher,VGC_Score,Critic_Score,User_Score,Total_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year
25079,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/70589/plants-vs-...,PSV,Sony Computer Entertainment,8.0,N/A,N/A,NaN,NaN,NaN,NaN,NaN,21st Feb 12
25080,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72058/plants-vs-...,PS3,Sony Computer Entertainment,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,08th Feb 11
25081,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72059/plants-vs-...,DS,PopCap Games,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,18th Jan 11
25082,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72060/plants-vs-...,OSX,PopCap Games,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,05th Mar 09
25083,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72062/plants-vs-...,And,Electronic Arts,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,14th Dec 11
25084,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72064/plants-vs-...,X360,PopCap Games,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,08th Sep 10
25085,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72066/plants-vs-...,DSi,PopCap Games,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,14th Mar 11
25086,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72069/plants-vs-...,And,Electronic Arts,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,14th Dec 11
25087,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72070/plants-vs-...,OSX,PopCap Games,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,05th May 09
25088,Misc,Plants vs. Zombies,https://www.vgchartz.com/game/72071/plants-vs-...,WinP,PopCap Games,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,22nd Jun 11


### Platform

In [1968]:
col = df_raw.Platform
print(f"""{col.name} 컬럼 정보
- 데이터타입        : {col.dtype}
- shape             : {col.shape}
- 결측치 갯수       : {col.isna().sum()}
- 중복값 갯수       : {col.duplicated().sum()}
- 유니크값 갯수     : {len(col.unique())}
""")

Platform 컬럼 정보
- 데이터타입        : object
- shape             : (62437,)
- 결측치 갯수       : 0
- 중복값 갯수       : 62357
- 유니크값 갯수     : 80



In [1969]:
df_raw.Platform.unique()

array(['Series', 'All', 'PS3', 'PS4', 'PS2', 'X360', 'PC', 'XOne', 'PS',
       'PSP', 'Wii', 'DS', '3DS', '2600', 'NS', 'iOS', 'NES', 'GC',
       'WiiU', 'XB', 'N64', 'GEN', 'GBA', 'GB', 'PSV', 'SNES', 'DC',
       'SAT', 'XBL', 'PSN', '5200', 'OSX', 'And', 'DSiW', 'Lynx', 'SCD',
       'Linux', 'MS', 'WW', 'ZXS', 'ACPC', 'Amig', '7800', 'VC', 'GG',
       'DSi', 'PCE', 'AJ', 'WinP', 'Mob', 'iQue', 'WS', 'NG', 'GIZ',
       '3DO', 'VB', 'GBC', 'Ouya', 'NGage', 'AST', 'MSD', 'S32X', 'PCFX',
       'XS', 'PS5', 'Int', 'CV', 'Arc', 'OR', 'CDi', 'C128', 'CD32',
       'C64', 'BRW', 'FMT', 'MSX', 'ApII', 'Aco', 'BBCM', 'TG16'],
      dtype=object)

##### 플랫폼 데이터프레임만들기

In [1970]:
# 웹스크래핑으로 만든 플랫폼 리스트1 불러오기
console_list1_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/console_list.csv'
console_list1 = pd.read_csv(console_list1_path)                 # 데이터 읽기

console_list1.drop(columns=['Unnamed: 0'], inplace=True)        # Unnamed: 0 컬럼 제거
console_list1.rename(columns={'0': 'pf_name'}, inplace=True)    # 컬럼 이름 변경
console_list1['pf_name'] = console_list1['pf_name'].str.strip() # 공백 제거

print(console_list1.shape)
console_list1

(90, 1)


,pf_name
0,3DO Interactive Multiplayer
1,Acorn Electron
2,All
3,Amiga
4,Amiga CD32
...,...
85,Xbox 360
86,XBox Live
87,Xbox One
88,Xbox Series


In [1971]:
# 웹스크래핑으로 만든 플랫폼 리스트2 불러오기
console_list2_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/console_list2.csv'
console_list2 = pd.read_csv(console_list2_path)                 # 데이터 읽기

console_list2.drop(columns=['Unnamed: 0'], inplace=True)        # Unnamed: 0 컬럼 제거
console_list2.rename(columns={'0' : 'pf_name'}, inplace=True)   # 컬럼 이름 변경
console_list2['pf_name'] = console_list2['pf_name'].str.strip() # 공백 제거

console_list2 = pd.DataFrame(console_list2['pf_name'].str.split('(', n=2, expand=True))
console_list2[1] = console_list2[1].str.slice(0,-1)
console_list2[0] = console_list2[0].str.strip()                 # 공백 제거
console_list2[1] = console_list2[1].str.strip()                 # 공백 제거
console_list2.rename(columns={0 : 'pf_name', 1 :'pf_nick'}, inplace=True) # 컬럼 이름 변경

print(console_list2.shape)
console_list2.head()

(51, 2)


,pf_name,pf_nick
0,PlayStation 2,PS2
1,Nintendo DS,DS
2,Nintendo Switch,NS
3,Game Boy,GB
4,PlayStation 4,PS4


In [1972]:
# 웹스크래핑으로 만든 플랫폼 리스트 합치기
console_list = pd.merge(left = console_list1, right = console_list2, how = 'left', on = 'pf_name')
console_list

,pf_name,pf_nick
0,3DO Interactive Multiplayer,3DO
1,Acorn Electron,NaN
2,All,NaN
3,Amiga,NaN
4,Amiga CD32,CD32
...,...,...
85,Xbox 360,X360
86,XBox Live,NaN
87,Xbox One,XOne
88,Xbox Series,NaN


In [1973]:
print(len(console_list.pf_nick.unique()))
console_list.pf_nick.unique()

46


array(['3DO', nan, 'CD32', '2600', '5200', '7800', 'AJ', 'Lynx', 'CDi',
       'CV', 'FCF', 'FDS', 'GB', 'GBA', 'GC', 'GG', 'GIZ', 'Int', 'Odys',
       'NGage', 'NG', '3DS', 'N64', 'DS', 'NES', 'NS', 'PCFX', 'PS',
       'PS2', 'PS3', 'PS4', 'PS5', 'PSP', 'PSV', 'S32X', 'SCD', 'GEN',
       'MS', 'SAT', 'SNES', 'TG16', 'VB', 'WS', 'XB', 'X360', 'XOne'],
      dtype=object)

###### 결측치 작업1

In [1974]:
console_list.isna().sum()

pf_name     0
pf_nick    45
dtype: int64

In [1975]:
isna_filt = console_list['pf_nick'].isna()

In [1976]:
console_list.loc[isna_filt].iloc[:10]

,pf_name,pf_nick
1,Acorn Electron,NaN
2,All,NaN
3,Amiga,NaN
5,Amstrad CPC,NaN
6,Android,NaN
7,Apple II,NaN
8,Arcade,NaN
14,Atari ST,NaN
15,BBC Micro,NaN
16,Browser,NaN


In [1977]:
column = 'pf_nick'

index = console_list.query('pf_name == "Acorn Electron"').index
console_list.loc[index, column] = 'Aco'

index = console_list.query('pf_name == "All"').index
console_list.loc[index, column] = 'All'

index = console_list.query('pf_name == "Amiga"').index
console_list.loc[index, column] = 'Amig'

index = console_list.query('pf_name == "Amstrad CPC"').index
console_list.loc[index, column] = 'ACPC'

index = console_list.query('pf_name == "Android"').index
console_list.loc[index, column] = 'And'

index = console_list.query('pf_name == "Apple II"').index
console_list.loc[index, column] = 'ApII'

index = console_list.query('pf_name == "Arcade"').index
console_list.loc[index, column] = 'Arc'

index = console_list.query('pf_name == "Atari ST"').index
console_list.loc[index, column] = 'AST'

index = console_list.query('pf_name == "BBC Micro"').index
console_list.loc[index, column] = 'BBCM'

index = console_list.query('pf_name == "Browser"').index
console_list.loc[index, column] = 'BRW'

index = console_list.query('pf_name == "Browser"').index
console_list.loc[index, column] = 'BRW'

index = console_list.query('pf_name == "Commodore 128"').index
console_list.loc[index, column] = 'C128'

index = console_list.query('pf_name == "Commodore 64"').index
console_list.loc[index, column] = 'C64'

index = console_list.query('pf_name == "DSi Ware"').index
console_list.loc[index, column] = 'DSiW'

index = console_list.query('pf_name == "FM Towns"').index
console_list.loc[index, column] = 'FMT'

index = console_list.query('pf_name == "Game Boy Color"').index
console_list.loc[index, column] = 'GBC'

index = console_list.query('pf_name == "iOS"').index
console_list.loc[index, column] = 'iOS'

index = console_list.query('pf_name == "iQue"').index
console_list.loc[index, column] = 'iQue'

index = console_list.query('pf_name == "Linux"').index
console_list.loc[index, column] = 'Linux'

index = console_list.query('pf_name == "Mac OS X"').index
console_list.loc[index, column] = 'OSX'

index = console_list.query('pf_name == "MS-DOS"').index
console_list.loc[index, column] = 'MSD'

index = console_list.query('pf_name == "MSX"').index
console_list.loc[index, column] = 'MSX'

index = console_list.query('pf_name == "Oculus Rift"').index
console_list.loc[index, column] = 'OR'

index = console_list.query('pf_name == "Nintendo DSi"').index
console_list.loc[index, column] = 'DSi'

index = console_list.query('pf_name == "Ouya"').index
console_list.loc[index, column] = 'Ouya'

index = console_list.query('pf_name == "PCE"').index
console_list.loc[index, column] = 'PCE'

index = console_list.query('pf_name == "PlayStation Network"').index
console_list.loc[index, column] = 'PSN'

index = console_list.query('pf_name == "Sega Dreamcast"').index
console_list.loc[index, column] = 'DC'

index = console_list.query('pf_name == "Series"').index
console_list.loc[index, column] = 'Series'

index = console_list.query('pf_name == "Standard Mobile"').index
console_list.loc[index, column] = 'Mob'

index = console_list.query('pf_name == "Virtual Console"').index
console_list.loc[index, column] = 'VC'

index = console_list.query('pf_name == "Wii"').index
console_list.loc[index, column] = 'Wii'

index = console_list.query('pf_name == "Wii U"').index
console_list.loc[index, column] = 'WiiU'

index = console_list.query('pf_name == "WiiWare"').index
console_list.loc[index, column] = 'WW'

index = console_list.query('pf_name == "Windows Phone"').index
console_list.loc[index, column] = 'WinP'

index = console_list.query('pf_name == "XBox Live"').index
console_list.loc[index, column] = 'XBL'

index = console_list.query('pf_name == "Xbox Series"').index
console_list.loc[index, column] = 'XS'

index = console_list.query('pf_name == "ZX Spectrum"').index
console_list.loc[index, column] = 'ZXS'

###### 결측치 작업2

In [1978]:
console_list.isna().sum()

pf_name    0
pf_nick    8
dtype: int64

In [1979]:
isna_filt = console_list['pf_nick'].isna()
console_list.loc[isna_filt]
# 해당 플랫폼들은 데이터가 없기 때문에 제거

,pf_name,pf_nick
31,HTC Vive,NaN
38,Microsoft Windows,NaN
43,Neo Geo X,NaN
44,Nintendo 2DS,NaN
51,Nvidia Shield,NaN
64,Playstation VR,NaN
73,Super Famicom,NaN
76,Unknown,NaN


In [1980]:
console_list = console_list.drop(index=console_list.loc[isna_filt].index) # 비어있는 플랫폼
console_list = console_list.sort_values('pf_name')                        # 정렬
console_list = console_list.reset_index(drop=True)                        # 인덱스 재지정

print('💚 console_list 결측치 💚\n', console_list.isna().sum(), '\n')
print('💚 console_list shape 💚\n', console_list.shape, '\n')
console_list

💚 console_list 결측치 💚
 pf_name    0
pf_nick    0
dtype: int64 

💚 console_list shape 💚
 (82, 2) 



,pf_name,pf_nick
0,3DO Interactive Multiplayer,3DO
1,Acorn Electron,Aco
2,All,All
3,Amiga,Amig
4,Amiga CD32,CD32
...,...,...
77,Xbox One,XOne
78,Xbox Series,XS
79,ZX Spectrum,ZXS
80,iOS,iOS


##### 플랫폼 데이터 비교

In [1981]:
isin_filt = console_list['pf_nick'].isin(df_raw.Platform)
console_list[~isin_filt]      # 빈 플랫폼은 삭제

,pf_name,pf_nick
23,Fairchild Channel F,FCF
24,Famicom Disk System,FDS
36,Magnavox Odyssey,Odys


In [1982]:
console_list = console_list.drop(index=console_list[~isin_filt].index)
console_list = console_list.reset_index(drop=True) # 인덱스 재지정
isin_filt = console_list['pf_nick'].isin(df_raw.Platform)
console_list[~isin_filt] # 빈 플랫폼 없음

,pf_name,pf_nick


In [ ]:
print(len(console_list['pf_nick'].unique()))
console_list['pf_nick'].unique()

In [ ]:
plat_list = pd.DataFrame(df_raw.Platform.unique())
plat_list

In [ ]:
isin_filt = console_list['pf_nick'].isin(plat_list[0])
console_list[~isin_filt]

In [ ]:
print(len(df_raw.Platform.value_counts().sort_index().index))
df_raw.Platform.value_counts().sort_index().index

In [ ]:
print(len(console_list['pf_nick'].unique()))
console_list['pf_nick'].unique()

In [ ]:
isin_filt = plat_list[0].isin(console_list['pf_nick'])
plat_list[~isin_filt]

In [ ]:
# console_list 에 PC 추가하기
index = len(console_list)+1
console_list.loc[index] = [ 'PC Games', 'PC' ]

In [ ]:
console_list.tail()

In [ ]:
console_list = console_list.sort_values('pf_name') # 정렬
console_list = console_list.reset_index(drop=True) # 인덱스 재지정

print('💚 console_list 결측치 💚\n', console_list.isna().sum(), '\n')
print('💚 console_list shape 💚\n', console_list.shape, '\n')
console_list

##### 플랫폼 세대분류
- [Home video game console generations](https://en.wikipedia.org/wiki/Home_video_game_console_generations)

<center>
<img alt="image" 
width=90% 
src="https://upload.wikimedia.org/wikipedia/en/timeline/dvi8hgniehwes0vxhfxxuhflggelxtl.png">  
</center>

In [ ]:
# 엑셀저장
console_list_xlsx_save_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/console_list_work.xlsx'
# console_list.to_excel(console_list_xlsx_save_path)

In [ ]:
console_list.pf_name.unique()

In [ ]:
console_list.pf_nick.unique()

In [ ]:
pf_gens = [5, 2, 8, 2, 5, 2, 8, 1, 1, 2, 
        2, 3, 5, 4, 2, 3, 7, 4, 2, 3, 
        2, 9, 5, 5, 6, 6, 6, 5, 6, 2, 
        8, 3, 2, 8, 6, 4, 7, 5, 6, 6, 
        3, 8, 9, 8, 8, 5, 3, 5, 6, 7, 
        8, 9, 7, 7, 7, 5, 4, 6, 4, 3, 
        4, 0, 8, 4, 4, 7, 7, 7, 8, 7, 
        8, 5, 6, 6, 7, 8, 9, 2, 8, 6]

In [ ]:
pf_manufacturer = ['Panasonic', 'Acorn', 'Unknown', 'Commodore', 'Commodore', 'Amstrad', 'Unknown', 'Apple', 'Unknown', 'Atari', 
                'Atari', 'Atari', 'Atari', 'Atari', 'Atari', 'Acorn', 'Unknown', 'Sony', 'Coleco', 'Commodore', 
                'Commodore', 'Nintendo', 'Fujitsu', 'Nintendo', 'Nintendo', 'Nintendo', 'Nintendo', 'Sega', 'Tiger Telematics', 'Mattel Electronics', 
                'Unknown', 'Unknown', 'ASCII', 'Unknown', 'Nokia', 'Neo Geo', 'Nintendo', 'Nintendo', 'Nintendo', 'Nintendo', 
                'Nintendo', 'Nintendo', 'Oculus', 'Ouya', 'Unknown', 'NEC', 'NEC', 'Sony', 'Sony', 'Sony', 
                'Sony', 'Sony', 'Sony', 'Sony', 'Sony', 'Sega', 'Sega', 'Sega', 'Sega', 'Sega', 
                'Sega', 'Unknown', 'Unknown', 'Nintendo', 'NEC', 'Nintendo', 'Nintendo', 'Nintendo', 'Nintendo', 'Nintendo', 
                'Microsoft', 'Bandai', 'Microsoft', 'Microsoft', 'Microsoft', 'Microsoft', 'Microsoft', 'Sinclair', '-', 'iQue']

In [ ]:
pf_release = ['1993', '1983', '-', '1985', '1993', '1984', '-', '1977', '-', '1977', 
              '1982', '1986', '1993', '1989', '1985', '1981', '-', '1990', '1982', '1985', 
              '1982', '2010', '1989', '1989', '2001', '1998', '2001', '1990', '2005', '1979 ', 
              '-', '-', '1983 ', '-', '2003', '1990', '2011', '1996', '2004', '2008', 
              '1983 ', '2017', '2012', '2013', '-', '1994', '1987', '1994', '2000', '2006 ', 
              '2013', '2020', '2006', '2004', '2011', '1994', '1991', '1998', '1988', '1985 ', 
              '1994', '-', '-', '1990', '1987', '1995', '2006', '2006', '2012', '2012', 
              '2010', '1999', '2002', '2001', '2005', '2013', '2020', '1982', '-', '2002']

In [ ]:
pf_type = ['video game console', 'computer', 'computer', 'computer', 'video game console', 'computer', 'mobile', 'computer', 'computer', 'video game console', 
           'video game console', 'video game console', 'video game console', 'Handheld', 'computer', 'computer', 'computer', 'video game console', 'video game console', 'computer', 
           'computer', 'Virtual Reality', 'video game console', 'Handheld', 'Handheld', 'Handheld', 'video game console', 'Handheld', 'Handheld', 'video game console', 
           'computer', 'computer', 'computer', 'computer', 'Handheld', 'video game console', 'Handheld', 'video game console', 'Handheld', 'Handheld', 
           'video game console', 'Handheld', 'Virtual Reality', 'video game console', 'computer', 'video game console', 'video game console', 'video game console', 'video game console', 'video game console', 
           'video game console', 'video game console', 'Online service', 'Handheld', 'Handheld', 'video game console', 'video game console', 'video game console', 'video game console', 'video game console', 
           'video game console', 'computer', 'mobile', 'video game console', 'video game console', 'Virtual Reality', 'Virtual Reality', 'video game console', 'Handheld', 'Virtual Reality', 
           'mobile', 'Handheld', 'Online service', 'video game console', 'video game console', 'video game console', 'video game console', 'computer', 'mobile', 'video game console']

In [ ]:
console_list['pf_gens'] = pf_gens
console_list['pf_manufacturer'] = pf_manufacturer
console_list['pf_release'] = pf_release
console_list['pf_type'] = pf_type

In [ ]:
console_list.pf_release.replace('-', '9999', inplace=True)
console_list.pf_release = console_list.pf_release.astype(int)

##### 기존 데이터프레임에 플랫폼 정보 merge

In [ ]:
df_raw.head()

In [ ]:
console_list.rename(columns={'pf_nick': 'Platform'}, inplace=True) # 컬럼 이름 변경

In [ ]:
console_list.head()

In [ ]:
df_raw = pd.merge(left = df_raw, right = console_list, how = 'left', on = 'Platform')

##### 플랫폼 범주화

In [ ]:
# 플랫폼 7 범주화
pc_7 = ['PC', 'OSX', 'Series', 'Linux', 'BRW', 'MSD', 'ACPC', 'MSX', 'ApII', 'All']
mobile_7 = ['And', 'iOS', 'WinP', 'Mob']
console_7 = ['PS2', 'PS', 'PS4', 'PS3', 'XOne', 'X360', 'Wii', 'SNES', 'NES', 'XB', 'GEN', 'SAT', 'VC', 'GC', 'DC', 
           '2600', 'PCE', 'XS', 'PS5', 'N64', 'MS', '3DO', 'SCD', 'NG', 'Int', 'DSi', 'AJ', '5200', 'PCFX', '7800', 'CV', 'C64', 
           'Amig', 'Ouya', 'ZXS', 'AST', 'iQue', 'CDi', 'S32X', 'CD32', 'FMT', 'TG16', 'C128', 'Aco', 'BBCM']
portable_7 = ['DS', 'NS', 'PSP', 'GBA', 'GB', '3DS', 'PSV', 'WiiU', 'GG', 'WS', 'Lynx', 'NGage', 'GIZ', 'GBC', ]
arcade_7 = ['Arc']
network_7 = ['XBL', 'PSN']
vr_7 = ['OR', 'VB', 'DSiW', 'WW']

In [ ]:
df_raw['Platform_7'] = 'pc'
df_raw.loc[df_raw[df_raw.Platform.isin(mobile_7)].index, 'Platform_7'] = 'mobile'
df_raw.loc[df_raw[df_raw.Platform.isin(console_7)].index, 'Platform_7'] = 'console'
df_raw.loc[df_raw[df_raw.Platform.isin(portable_7)].index, 'Platform_7'] = 'portable'
df_raw.loc[df_raw[df_raw.Platform.isin(arcade_7)].index, 'Platform_7'] = 'arcade'
df_raw.loc[df_raw[df_raw.Platform.isin(network_7)].index, 'Platform_7'] = 'network'
df_raw.loc[df_raw[df_raw.Platform.isin(vr_7)].index, 'Platform_7'] = 'vr'

In [ ]:
print('Platform_7 결측치 : ', df_raw['Platform_7'].isna().sum(), '\n')
df_raw['Platform_7'].value_counts()

In [ ]:
# 플랫폼 3 범주화
pc_3 = ['PC', 'OSX', 'Series', 'Linux', 'BRW', 'MSD', 'ACPC', 'MSX', 'ApII', 'All']
console_3 = ['PS2', 'PS', 'PS4', 'PS3', 'XOne', 'X360', 'Wii', 'SNES', 'NES', 'XB', 'GEN', 'SAT', 'VC', 'GC', 'DC', 'XBL', 'PSN', 'Arc',
           '2600', 'PCE', 'XS', 'PS5', 'N64', 'MS', '3DO', 'SCD', 'NG', 'Int', 'DSi', 'AJ', '5200', 'PCFX', '7800', 'CV', 'C64', 
           'Amig', 'Ouya', 'ZXS', 'AST', 'iQue', 'CDi', 'S32X', 'CD32', 'FMT', 'TG16', 'C128', 'Aco', 'BBCM', 'OR', 'VB', 'DSiW', 'WW']
portable_3 = ['DS', 'NS', 'PSP', 'GBA', 'GB', '3DS', 'PSV', 'WiiU', 'GG', 'WS', 'Lynx', 'NGage', 'GIZ', 'GBC', 'And', 'iOS', 'WinP', 'Mob']

In [ ]:
df_raw['Platform_3'] = 'pc'
df_raw.loc[df_raw[df_raw.Platform.isin(console_3)].index, 'Platform_3'] = 'console'
df_raw.loc[df_raw[df_raw.Platform.isin(portable_3)].index, 'Platform_3'] = 'portable'

In [ ]:
print('Platform_3 결측치 : ', df_raw['Platform_3'].isna().sum(), '\n')
df_raw['Platform_3'].value_counts()

### Year

In [ ]:
col = df_raw.Year
print(f"""{col.name} 컬럼 정보
- 데이터타입        : {col.dtype}
- shape             : {col.shape}
- 결측치 갯수       : {col.isna().sum()}
- 중복값 갯수       : {col.duplicated().sum()}
- 유니크값 갯수     : {len(col.unique())}
""")

In [ ]:
# data 확인
df_raw.Year.value_counts()

In [ ]:
df_raw.Year = df_raw.Year.str.strip()

In [ ]:
df_raw.Year = df_raw.Year.str.slice(-2,) 

In [ ]:
# 결측데이터 보기
df_raw.Year.isna().sum() # 결측치 없음

In [ ]:
df_raw.loc[df_raw[df_raw.Year == '/A'].index, 'Year'] = '9999'

In [ ]:
df_raw.Year.value_counts()

In [ ]:
df_raw.Year = df_raw.Year.astype(int) # 데이터타입 정수형으로 변환
df_raw.Year.dtypes

In [ ]:
print(f"""\
출시년도가 40 이하인 케이스 수    : {df_raw.query('Year <= 40').shape[0]}
40 < 출시년도 <=99 케이스 수      : {df_raw.query('Year > 40 & Year <= 99').shape[0]}
""")

In [ ]:
df_raw.columns

In [ ]:
filt_40 = df_raw.Year <= 40 # 출시년도가 40 이하인 케이스 필터링
df_raw[filt_40].index
df_raw.loc[df_raw[filt_40].index, 'Year'] = df_raw.loc[df_raw[filt_40].index, 'Year'] + 2000 # 기존값에 +2000
df_raw[filt_40].Year.value_counts() # 출시년도가 40 이하인 케이스 +2000 완료

In [ ]:
filt_99 = df_raw.Year <= 99 # 출시년도가 99 이하인 케이스 필터링
df_raw[filt_99].index
df_raw.loc[df_raw[filt_99].index, 'Year'] = df_raw.loc[df_raw[filt_99].index, 'Year'] + 1900 # 기존값에 +1900
df_raw[filt_99].Year.value_counts() # 출시년도가 30 이하인 케이스 +1900 완료

In [ ]:
df_raw.Year.unique()

In [ ]:
filt = df_raw.Year < 1983
df_raw[filt].shape

### Genre

In [ ]:
col = df_raw.Genre
print(f"""{col.name} 컬럼 정보
- 데이터타입        : {col.dtype}
- shape             : {col.shape}
- 결측치 갯수       : {col.isna().sum()}
- 중복값 갯수       : {col.duplicated().sum()}
- 유니크값 갯수     : {len(col.unique())}
""")

##### 장르 : 범주 축소

- action = 'Action','Fighting','Shooter'
- rpg = 'MMO','Party','Role-Playing','Sandbox'
- adventure = 'Action-Adventure','Adventure','Platform','Visual Novel'
- simul = 'Simulation','Strategy'
- sports = 'Racing','Sports'
- misc = 'Board Game','Education','Misc','Music','Puzzle'

In [ ]:
action = ['Action','Fighting','Shooter']
rpg = ['MMO','Party','Role-Playing','Sandbox']
adventure = ['Action-Adventure','Adventure','Platform','Visual Novel']
simul = ['Simulation','Strategy']
sports = ['Racing','Sports']
misc = ['Board Game','Education','Misc','Music','Puzzle']

In [ ]:
df_raw['gen_6'] = 'action'
df_raw.loc[df_raw[df_raw.Genre.isin(rpg)].index, 'gen_6'] = 'rpg'
df_raw.loc[df_raw[df_raw.Genre.isin(adventure)].index, 'gen_6'] = 'adventure'
df_raw.loc[df_raw[df_raw.Genre.isin(simul)].index, 'gen_6'] = 'simul'
df_raw.loc[df_raw[df_raw.Genre.isin(sports)].index, 'gen_6'] = 'sports'
df_raw.loc[df_raw[df_raw.Genre.isin(misc)].index, 'gen_6'] = 'misc'

In [ ]:
df_raw['gen_6'].value_counts()

### Publisher

In [ ]:
col = df_raw.Publisher
print(f"""{col.name} 컬럼 정보
- 데이터타입        : {col.dtype}
- shape             : {col.shape}
- 결측치 갯수       : {col.isna().sum()}
- 중복값 갯수       : {col.duplicated().sum()}
- 유니크값 갯수     : {len(col.unique())}
""")

### XX_Score

In [ ]:
col = df_raw.VGC_Score
print(f"""{col.name} 컬럼 정보
- 데이터타입        : {col.dtype}
- shape             : {col.shape}
- 결측치 갯수       : {col.isna().sum()}
- 중복값 갯수       : {col.duplicated().sum()}
- 유니크값 갯수     : {len(col.unique())}
""")

In [ ]:
df_raw.VGC_Score.value_counts()

In [ ]:
df_raw.VGC_Score = df_raw.VGC_Score.str.strip() # 좌우 공백 제거

In [ ]:
# 다른 점수로 대체
df_raw.loc[(df_raw.VGC_Score.str.contains('N/A')) & (df_raw.Critic_Score.str.contains('[09]')), 'VGC_Score'] = df_raw['Critic_Score']
df_raw.loc[(df_raw.VGC_Score.str.contains('N/A')) & (df_raw.User_Score.str.contains('[09]')), 'VGC_Score'] = df_raw['User_Score']
df_raw.VGC_Score.value_counts() # 2034개 대체 됨

In [ ]:
col = df_raw.Critic_Score
print(f"""{col.name} 컬럼 정보
- 데이터타입        : {col.dtype}
- shape             : {col.shape}
- 결측치 갯수       : {col.isna().sum()}
- 중복값 갯수       : {col.duplicated().sum()}
- 유니크값 갯수     : {len(col.unique())}
""")

In [ ]:
df_raw.Critic_Score.value_counts()

In [ ]:
df_raw.Critic_Score = df_raw.Critic_Score.str.strip() # 좌우 공백 제거

In [ ]:
# 다른 점수로 대체
df_raw.loc[(df_raw.Critic_Score.str.contains('N/A')) & (df_raw.VGC_Score.str.contains('[09]')), 'Critic_Score'] = df_raw['VGC_Score']
df_raw.loc[(df_raw.Critic_Score.str.contains('N/A')) & (df_raw.User_Score.str.contains('[09]')), 'Critic_Score'] = df_raw['User_Score']
df_raw.Critic_Score.value_counts() # 830개 대체됨

In [ ]:
col = df_raw.User_Score
print(f"""{col.name} 컬럼 정보
- 데이터타입        : {col.dtype}
- shape             : {col.shape}
- 결측치 갯수       : {col.isna().sum()}
- 중복값 갯수       : {col.duplicated().sum()}
- 유니크값 갯수     : {len(col.unique())}
""")

In [ ]:
df_raw.User_Score.value_counts()

In [ ]:
df_raw.User_Score = df_raw.User_Score.str.strip() # 좌우 공백 제거

In [ ]:
62033 - 59032

In [ ]:
# 다른 점수로 대체
df_raw.loc[(df_raw.User_Score.str.contains('N/A')) & (df_raw.VGC_Score.str.contains('[09]')), 'User_Score'] = df_raw['VGC_Score']
df_raw.loc[(df_raw.User_Score.str.contains('N/A')) & (df_raw.Critic_Score.str.contains('[09]')), 'User_Score'] = df_raw['Critic_Score']
df_raw.User_Score.value_counts() # 3001개 대체 됨

In [ ]:
# 남아있는 N/A 5.55 로 대체
df_raw.replace('N/A', 5.55, inplace=True)

In [ ]:
# float 변환
df_raw['VGC_Score'] = df_raw['VGC_Score'].astype(float)
df_raw['Critic_Score'] = df_raw['Critic_Score'].astype(float)
df_raw['User_Score'] = df_raw['User_Score'].astype(float)

In [ ]:
df_raw[['VGC_Score', 'Critic_Score', 'User_Score']].info()

In [ ]:
# Score 변수 만들기
df_raw['Score'] = (df_raw.VGC_Score + df_raw.Critic_Score + df_raw.User_Score) / 3

### XX_Sales

In [ ]:
df_raw.columns

In [ ]:
region_list = ['NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales']

In [ ]:
df_raw[region_list].info()

In [ ]:
for i in region_list:
  print(i, ' : ', df_raw[i].str.contains(pat='[^\d]', regex=True).sum()) # 숫자가 아닌것

In [ ]:
for i in region_list:
  print(df_raw[i].str.contains(pat='[m]', regex=True).sum()) # m이 있는 것

In [ ]:
for i in region_list:
  index_m = df_raw[i].str.contains(pat='[m]', regex=True).index   # m이 붙어있는 것들의 인덱스 저장
  df_raw[i].replace('[a-zA-Z]', '', regex=True, inplace=True)     # 문자 제거
  df_raw[i] = df_raw[i].astype(float)                             # 테이터타입 소수형 변형
  df_raw.loc[index_m, i] = df_raw.loc[index_m, i] * 1000000       # 기존값에 x 1 million

In [ ]:
df_raw[region_list].describe()

In [ ]:
df_raw[region_list].shape

In [ ]:
# 'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales' 모두 결측인 케이스
filt_na = df_raw.NA_Sales.isna() & df_raw.PAL_Sales.isna() & df_raw.JP_Sales.isna() & df_raw.Other_Sales.isna()
df_raw[filt_na].shape

##### rc_df (Region Cleaned Data Frame)

In [ ]:
df_raw.columns

In [ ]:
# 모든 지역의 판매량이 결측인 데이터 제거 = Region Cleaned Data Frame (rc_df)
rc_df = df_raw.drop(index=df_raw[filt_na].index)

In [ ]:
rc_df[region_list].isna().sum()

In [ ]:
# 남아있는 결측은 0으로 대체
for region in region_list:
  rc_df[region].fillna(0, inplace=True)

In [ ]:
rc_df[region_list].isna().sum()

In [ ]:
print(rc_df.shape)
rc_df[region_list].describe()

In [ ]:
# 'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales' 를 롱타입으로 변환
df = pd.melt(rc_df, id_vars=['Genre', 'Name', 'url', 'Platform', 'Publisher', 'Score', 'VGC_Score',\
                                'Critic_Score', 'User_Score', 'Total_Sales', 'Year', 'pf_name', 'pf_gens', 'pf_manufacturer', \
                                'pf_release', 'pf_type', 'Platform_7', 'Platform_3', 'gen_6'],
                        var_name='region', value_name='sales') 

In [ ]:
# 지역명에서 _sales 제거
df.region = df.region.str.slice(0,-6)
df.head()

In [ ]:
# 판매량이 0인 케이스 제거
filt = df.sales == 0
df.drop(index=df[filt].index, inplace=True)
filt = df.sales == 0
df['sales'] = df['sales'].astype(int)         # 데이터 타입 정수형으로 변환
df[filt].shape

In [ ]:
df.shape

##### Total_Sales

In [ ]:
rc_df.Total_Sales.str.contains(pat='[^\d]', regex=True).sum() # 숫자가 아닌것

In [ ]:
rc_df.Total_Sales.str.contains(pat='[m]', regex=True).sum() # m이 있는 것

In [ ]:
index_m = rc_df.Total_Sales.str.contains(pat='[m]', regex=True).index        # m이 붙어있는 것들의 인덱스 저장
rc_df.Total_Sales.replace('[a-zA-Z]', '', regex=True, inplace=True)          # 문자 제거
rc_df.Total_Sales = rc_df.Total_Sales.astype(float)                             # 테이터타입 소수형 변형
rc_df.loc[index_m, 'Total_Sales'] = rc_df.loc[index_m, 'Total_Sales'] * 1000000 # 기존값에 x 1 million
rc_df.Total_Sales.describe()

In [ ]:
# totla sales 데이터프레임 저장
df_total = rc_df[['Genre', 'Name', 'url', 'Platform', 'Publisher', 'Score', 'VGC_Score',\
       'Critic_Score', 'User_Score', 'Total_Sales', 'Year', 'pf_name',\
       'pf_gens', 'pf_manufacturer', 'pf_release', 'pf_type', 'Platform_7',\
       'Platform_3', 'gen_6']]

In [ ]:
print(df_total.shape)
df_total.head()

## 분석

### 지역에 따른 평균 비디오게임 판매량의 차이

In [ ]:
Count = df.groupby('region').count()['sales']
Total = df.groupby('region').sum()['sales']
Mean = df.groupby('region').mean()['sales']

data = {
    'Count' : Count,
    'Total' : Total,
    'Mean' : Mean,
}

results = pd.DataFrame(data)

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_sale_summary.xlsx'    # 저장경로
result.to_excel(path)                                                                                   # 엑셀 저장
results

In [ ]:
# box plot을 그리기 위한 그룹 지정
group1 = df.query('region == "NA"')['sales']
group2 = df.query('region == "PAL"')['sales']
group3 = df.query('region == "JP"')['sales']
group4 = df.query('region == "Other"')['sales']

plot_data = [group1, group2, group3, group4]

# 그래프 그리기
fig, ax = plt.subplots(figsize=(15, 7))

ax = plt.boxplot(plot_data, notch=True, whis=2.5)
plt.ylim(-50000, 500000)

plt.title('Box plot', fontsize=24, fontweight = 'bold', pad=20)

plt.xlabel('Groups', fontsize=14)
plt.ylabel('Variation', fontsize=14)

plt.text(1, -40000, 'Group : NA', ha='center')
plt.text(2, -40000, 'Group : PAL', ha='center')
plt.text(3, -40000, 'Group : JP', ha='center')
plt.text(4, -40000, 'Group : Other', ha='center')

medians = [item.get_ydata() for item in ax['medians']]

plt.show()

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_sale_01_boxplot.png'
fig.savefig(path, dpi=150, facecolor='#eeeeee')

In [ ]:
F_statistic, pVal = stats.f_oneway(group1, group2, group3, group4)      # 일원분산분석
posthoc = pairwise_tukeyhsd(df.sales, df.region, alpha=0.05)            # 사후검정 : tukey HSD

results = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])  # 분석결과

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_sale_02_oneway_anova.xlsx'
results.to_excel(path)  # 분석결과 엑셀 저장

print(f"""일원분산분석 결과
F = {F_statistic:.3f}, p = {pVal:.3f}

{results}
""")

In [ ]:
fig = posthoc.plot_simultaneous() # 사후검정 시각화 그래프

plt.xlabel('Groups')
plt.ylabel('Variation')

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_sale_03_posthoc.png'
fig.savefig(path, dpi=150, facecolor='#eeeeee')

In [ ]:
filt_NA = df.region == "NA"
filt_PAL = df.region == "PAL"
filt_JP = df.region == "JP"
filt_Other = df.region == "Other"
filt_year = df.Year == 9999

## 그래프 그리기
fig, ax = plt.subplots(figsize=(13, 5))

# NA 그래프
x_NA = df[~filt_year][filt_NA].groupby('Year').sum()['sales'].index
y_NA = df[~filt_year][filt_NA].groupby('Year').sum()['sales'].values
ax.plot(x_NA, y_NA, label='NA')

# PAL 그래프
x_PAL = df[~filt_year][filt_PAL].groupby('Year').sum()['sales'].index
y_PAL = df[~filt_year][filt_PAL].groupby('Year').sum()['sales'].values
ax.plot(x_PAL, y_PAL, label='PAL')

# JP 그래프
x_JP = df[~filt_year][filt_JP].groupby('Year').sum()['sales'].index
y_JP = df[~filt_year][filt_JP].groupby('Year').sum()['sales'].values
ax.plot(x_JP, y_JP, label='JP')

# Other 그래프
x_Other = df[~filt_year][filt_Other].groupby('Year').sum()['sales'].index
y_Other = df[~filt_year][filt_Other].groupby('Year').sum()['sales'].values
ax.plot(x_Other, y_Other, label='Other')

## 그래프 설정
plt.title('Sales by Region and Year', fontsize=22, fontweight = 'bold', pad=15)

plt.xlabel('Year', fontsize=14)
plt.ylabel('Sales', fontsize=14)

plt.xlim(1970, 2021)
ax.legend()
plt.show()

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_sale_04_Sales_by_Region_and_Year.png'
fig.savefig(path, dpi=150, facecolor='#eeeeee')

In [ ]:
# 2012년 이후 분산분석
filt_year = (df.Year > 2012) & (df.Year < 3000)
group1 = df[filt_year].query('region == "NA"')['sales']
group2 = df[filt_year].query('region == "PAL"')['sales']
group3 = df[filt_year].query('region == "JP"')['sales']
group4 = df[filt_year].query('region == "Other"')['sales']

F_statistic, pVal = stats.f_oneway(group1, group2, group3, group4)
posthoc = pairwise_tukeyhsd(df[filt_year].sales, df[filt_year].region, alpha=0.05)

posthoc_df = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])
path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_sale_05_posthoc_up2012.xlsx'
posthoc_df.to_excel(path)

print(f"""일원분산분석 결과
F = {F_statistic:.3f}, p = {pVal:.3f}

{posthoc_df}
""")

### 지역에따라 선호하는 게임의 장르가 있을까?

In [ ]:
filt_NA = df.region == "NA"
filt_PAL = df.region == "PAL"
filt_JP = df.region == "JP"
filt_Other = df.region == "Other"

values_NA = df[filt_NA].groupby('gen_6').sum()['sales'].values
labels_NA = df[filt_NA].groupby('gen_6').sum()['sales'].index

values_PAL = df[filt_PAL].groupby('gen_6').sum()['sales'].values
labels_PAL = df[filt_PAL].groupby('gen_6').sum()['sales'].index

values_JP = df[filt_JP].groupby('gen_6').sum()['sales'].values
labels_JP = df[filt_JP].groupby('gen_6').sum()['sales'].index

values_Other = df[filt_Other].groupby('gen_6').sum()['sales'].values
labels_Other = df[filt_Other].groupby('gen_6').sum()['sales'].index

## 그래프 그리기
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,12))

ax[0][0].pie(values_NA, labels=labels_NA,
    autopct='%.1f%%',
    startangle=90,
    colors = sns.color_palette('pastel')[0:6],
    wedgeprops={'width':0.6, 'edgecolor':'w', 'linewidth':2},
    counterclock=False)
ax[0][1].pie(values_PAL, labels=labels_PAL, 
    autopct='%.1f%%', 
    startangle=90, 
    colors = sns.color_palette('pastel')[0:6],
    wedgeprops={'width':0.6, 'edgecolor':'w', 'linewidth':2},
    counterclock=False)
ax[1][0].pie(values_JP, labels=labels_JP, 
    autopct='%.1f%%', 
    startangle=90, 
    colors = sns.color_palette('pastel')[0:6],
    wedgeprops={'width':0.6, 'edgecolor':'w', 'linewidth':2},
    counterclock=False)
ax[1][1].pie(values_Other, labels=labels_Other, 
    autopct='%.1f%%', 
    startangle=90, 
    colors = sns.color_palette('pastel')[0:6],
    wedgeprops={'width':0.6, 'edgecolor':'w', 'linewidth':2},
    counterclock=False)

ax[0][0].set_title('NA', fontsize=20, fontweight = 'bold', y=0.5)
ax[0][1].set_title('PAL', fontsize=20, fontweight = 'bold', y=0.5)
ax[1][0].set_title('JP', fontsize=20, fontweight = 'bold', y=0.5)
ax[1][1].set_title('Other', fontsize=20, fontweight = 'bold', y=0.5)


## 그래프 설정
fig.suptitle('Rate of Genre by Region', fontsize=28, fontweight = 'bold', y=0.95)
# fig.subplots_adjust(top=0.8)

plt.show()


path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_01_region_genre.png'
fig.savefig(path, dpi=150, facecolor='#eeeeee')

In [ ]:
# NA 지역의 비디오게임 장르 분포
region = filt_NA
total_sale = df[region].sum()['sales']
group1 = df[region].query('gen_6 == "action"')['sales']
group2 = df[region].query('gen_6 == "adventure"')['sales']
group3 = df[region].query('gen_6 == "misc"')['sales']
group4 = df[region].query('gen_6 == "rpg"')['sales']
group5 = df[region].query('gen_6 == "sports"')['sales']
group6 = df[region].query('gen_6 == "simul"')['sales']

idx = ['action','adventure','misc','rpg','sports','simul']
data = {
    'Sales' : [group1.sum(), group2.sum(), group3.sum(), group4.sum(), group5.sum(), group6.sum()],
    'Sales_Mean' : [group1.mean(), group2.mean(), group3.mean(), group4.mean(), group5.mean(), group6.mean()],
    'Rate' : [group1.sum() / total_sale, group2.sum() / total_sale, group3.sum() / total_sale, group4.sum() / total_sale, group5.sum() / total_sale, group6.sum() / total_sale],
}

summary_table = pd.DataFrame(data, index=idx)

F_statistic, pVal = stats.f_oneway(group1, group2, group3, group4, group5, group6)
posthoc = pairwise_tukeyhsd(df[region].sales, df[region].gen_6, alpha=0.05)

posthoc_df = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])
path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_02_na_genre.xlsx'
posthoc_df.to_excel(path)

print(f"""
{summary_table.sort_values(['Rate'], ascending=False)}

일원분산분석 결과
F = {F_statistic:.3f}, p = {pVal:.3f}

{posthoc_df}
""")

In [ ]:
# PAL 지역의 비디오게임 장르 분포
region = filt_PAL
total_sale = df[region].sum()['sales']
group1 = df[region].query('gen_6 == "action"')['sales']
group2 = df[region].query('gen_6 == "adventure"')['sales']
group3 = df[region].query('gen_6 == "misc"')['sales']
group4 = df[region].query('gen_6 == "rpg"')['sales']
group5 = df[region].query('gen_6 == "sports"')['sales']
group6 = df[region].query('gen_6 == "simul"')['sales']

idx = ['action','adventure','misc','rpg','sports','simul']
data = {
    'Sales' : [group1.sum(), group2.sum(), group3.sum(), group4.sum(), group5.sum(), group6.sum()],
    'Rate' : [group1.sum() / total_sale, group2.sum() / total_sale, group3.sum() / total_sale, group4.sum() / total_sale, group5.sum() / total_sale, group6.sum() / total_sale],
}

summary_table = pd.DataFrame(data, index=idx)

F_statistic, pVal = stats.f_oneway(group1, group2, group3, group4, group5, group6)
posthoc = pairwise_tukeyhsd(df[region].sales, df[region].gen_6, alpha=0.05)

posthoc_df = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])
path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_03_pal_genre.xlsx'
posthoc_df.to_excel(path)

print(f"""
{summary_table.sort_values(['Rate'], ascending=False)}

일원분산분석 결과
F = {F_statistic:.3f}, p = {pVal:.3f}

{posthoc}
""")

In [ ]:
# JP 지역의 비디오게임 장르 분포
region = filt_JP
total_sale = df[region].sum()['sales']
group1 = df[region].query('gen_6 == "action"')['sales']
group2 = df[region].query('gen_6 == "adventure"')['sales']
group3 = df[region].query('gen_6 == "misc"')['sales']
group4 = df[region].query('gen_6 == "rpg"')['sales']
group5 = df[region].query('gen_6 == "sports"')['sales']
group6 = df[region].query('gen_6 == "simul"')['sales']

idx = ['action','adventure','misc','rpg','sports','simul']
data = {
    'Sales' : [group1.sum(), group2.sum(), group3.sum(), group4.sum(), group5.sum(), group6.sum()],
    'Rate' : [group1.sum() / total_sale, group2.sum() / total_sale, group3.sum() / total_sale, group4.sum() / total_sale, group5.sum() / total_sale, group6.sum() / total_sale],
}

summary_table = pd.DataFrame(data, index=idx)

F_statistic, pVal = stats.f_oneway(group1, group2, group3, group4, group5, group6)
posthoc = pairwise_tukeyhsd(df[region].sales, df[region].gen_6, alpha=0.05)

posthoc_df = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])
path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_04_jp_genre.xlsx'
posthoc_df.to_excel(path)

print(f"""
{summary_table.sort_values(['Rate'], ascending=False)}

일원분산분석 결과
F = {F_statistic:.3f}, p = {pVal:.3f}

{posthoc}
""")

In [ ]:
# Other 지역의 비디오게임 장르 분포
region = filt_Other
total_sale = df[region].sum()['sales']
group1 = df[region].query('gen_6 == "action"')['sales']
group2 = df[region].query('gen_6 == "adventure"')['sales']
group3 = df[region].query('gen_6 == "misc"')['sales']
group4 = df[region].query('gen_6 == "rpg"')['sales']
group5 = df[region].query('gen_6 == "sports"')['sales']
group6 = df[region].query('gen_6 == "simul"')['sales']

idx = ['action','adventure','misc','rpg','sports','simul']
data = {
    'Sales' : [group1.sum(), group2.sum(), group3.sum(), group4.sum(), group5.sum(), group6.sum()],
    'Rate' : [group1.sum() / total_sale, group2.sum() / total_sale, group3.sum() / total_sale, group4.sum() / total_sale, group5.sum() / total_sale, group6.sum() / total_sale],
}

summary_table = pd.DataFrame(data, index=idx)

F_statistic, pVal = stats.f_oneway(group1, group2, group3, group4, group5, group6)
posthoc = pairwise_tukeyhsd(df[region].sales, df[region].gen_6, alpha=0.05)

posthoc_df = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])
path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_05_other_genre.xlsx'
posthoc_df.to_excel(path)

print(f"""
{summary_table.sort_values(['Rate'], ascending=False)}

일원분산분석 결과
F = {F_statistic:.3f}, p = {pVal:.3f}

{posthoc}
""")

In [ ]:
# 이원분산분석(two-way ANOVA) : statmodels
formula = 'sales ~ C(region) + C(gen_6) + C(region):C(gen_6)'
lm = ols(formula, df).fit()
results = anova_lm(lm, typ=3)

# posthoc_df = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])
path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_06_twoway_genre_region.xlsx'
results.to_excel(path)

print(results)

In [ ]:
fig = interaction_plot(df.gen_6, df.region, df.sales)

이원분산분석 + 사후검정
[ANOVA + Tukey Test In Python](https://towardsdatascience.com/anova-tukey-test-in-python-b3082b6e6bda)

In [ ]:
df['combination'] = df.region + ' / ' + df.gen_6 # 상호작용항 만들기
posthoc = pairwise_tukeyhsd(df.sales, df.combination, alpha=0.05)

In [ ]:
posthoc_df = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])
posthoc_df

In [ ]:
posthoc_df['group1_reg'] = posthoc_df['group1'].str.split(' / ', n=2, expand=True)[0]
posthoc_df['group1_gen'] = posthoc_df['group1'].str.split(' / ', n=2, expand=True)[1]
posthoc_df['group2_reg'] = posthoc_df['group2'].str.split(' / ', n=2, expand=True)[0]
posthoc_df['group2_gen'] = posthoc_df['group2'].str.split(' / ', n=2, expand=True)[1]

In [ ]:
posthoc_df = posthoc_df[['group1_reg', 'group1_gen', 'group2_reg', 'group2_gen', 'meandiff', 'p-adj', 'lower', 'upper', 'reject']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_07_twoway_posthoc.xlsx'
posthoc_df.to_excel(path)

posthoc_df

In [ ]:
filter1 = posthoc_df.group1_gen == "action"
filter2 = posthoc_df.group2_gen == "action"
results = posthoc_df[filter1][filter2]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_08_action.xlsx'
results.to_excel(path)
results

In [ ]:
filter1 = posthoc_df.group1_gen == "adventure"
filter2 = posthoc_df.group2_gen == "adventure"
results = posthoc_df[filter1][filter2]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_09_adventure.xlsx'
results.to_excel(path)
results

In [ ]:
filter1 = posthoc_df.group1_gen == "misc"
filter2 = posthoc_df.group2_gen == "misc"
results = posthoc_df[filter1][filter2]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_10_misc.xlsx'
results.to_excel(path)
results

In [ ]:
filter1 = posthoc_df.group1_gen == "rpg"
filter2 = posthoc_df.group2_gen == "rpg"
results = posthoc_df[filter1][filter2]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_11_rpg.xlsx'
results.to_excel(path)
results

In [ ]:
filter1 = posthoc_df.group1_gen == "simul"
filter2 = posthoc_df.group2_gen == "simul"
results = posthoc_df[filter1][filter2]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_12_simul.xlsx'
results.to_excel(path)
results

In [ ]:
filter1 = posthoc_df.group1_gen == "sports"
filter2 = posthoc_df.group2_gen == "sports"
results = posthoc_df[filter1][filter2]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/region_genre_13_sports.xlsx'
results.to_excel(path)
results

### 연도에 따라 비디오 게임의 인기 플랫폼이 다른가?

In [ ]:
filt_year = df_total.Year == 9999

## 그래프 그리기
fig, ax = plt.subplots(figsize=(13, 5))

for pf in [5,6,7,8]:
  x = df_total[~filt_year][df_total.pf_gens == pf].groupby('Year').sum()['Total_Sales'].index
  y = df_total[~filt_year][df_total.pf_gens == pf].groupby('Year').sum()['Total_Sales'].values
  ax.plot(x, y, lw=4, label=pf)

plt.text(1998, 10**8*0.8, '5 Generation', ha='center', fontsize=14)
plt.text(2003, 10**8*1.9, '6 Generation', ha='center', fontsize=14)
plt.text(2009, 10**8*3, '7 Generation', ha='center', fontsize=14)
plt.text(2016, 10**8*1.1, '8 Generation', ha='center', fontsize=14)

plt.title('Sales of Generation by Year', fontsize=22, fontweight = 'bold', pad=15)

plt.xlabel('Year')
plt.ylabel('Sales')

plt.xlim(1990, 2021)          

ax.legend()
plt.show()

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/year_platform_01_generation.png'
fig.savefig(path, dpi=150, facecolor='#eeeeee')

In [ ]:
result = df_total.sort_values('Total_Sales', ascending=False).head(10)[['Name','Total_Sales','pf_gens','Genre','Platform','Year']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/year_platform_02_all.xlsx'
result.to_excel(path)

result

In [ ]:
filt = df_total.pf_gens == 5
result = df_total[filt].sort_values('Total_Sales', ascending=False).head()[['Name','Total_Sales','pf_gens','Genre','Platform','Year']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/year_platform_03_5gen.xlsx'
result.to_excel(path)

result

In [ ]:
filt = df_total.pf_gens == 6
result = df_total[filt].sort_values('Total_Sales', ascending=False).head()[['Name','Total_Sales','pf_gens','Genre','Platform','Year']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/year_platform_04_6gen.xlsx'
result.to_excel(path)

result

In [ ]:
filt = df_total.pf_gens == 7
result = df_total[filt].sort_values('Total_Sales', ascending=False).head()[['Name','Total_Sales','pf_gens','Genre','Platform','Year']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/year_platform_05_7gen.xlsx'
result.to_excel(path)

result

In [ ]:
filt = df_total.pf_gens == 8
result = df_total[filt].sort_values('Total_Sales', ascending=False).head()[['Name','Total_Sales','pf_gens','Genre','Platform','Year']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/year_platform_06_8gen.xlsx'
result.to_excel(path)

result

### 인기가 많은 게임은 무엇?

In [ ]:
## 그래프 그리기
fig, ax = plt.subplots(figsize=(10, 7))

plt.scatter(df.Score, df.sales, alpha=0.5)

## 그래프 설정
plt.title('Sales by Score', fontsize=22, fontweight = 'bold', pad=15)


plt.xlabel('Score', fontsize=14)
plt.ylabel('Sales', fontsize=14)

plt.show()

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/pop_game_01_sales_score.png'
fig.savefig(path, dpi=150, facecolor='#eeeeee')

In [ ]:
filt_NA = df.region == "NA"
result = df[filt_NA].sort_values('sales', ascending=False).head()[['Name','Genre','Platform','Year','sales']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/pop_game_02_sales_na.xlsx'
result.to_excel(path)

result

In [ ]:
filt_PAL = df.region == "PAL"
result = df[filt_PAL].sort_values('sales', ascending=False).head()[['Name','Genre','Platform','Year','sales']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/pop_game_03_sales_pal.xlsx'
result.to_excel(path)

result

In [ ]:
filt_JP = df.region == "JP"
result = df[filt_JP].sort_values('sales', ascending=False).head()[['Name','Genre','Platform','Year','sales']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/pop_game_04_sales_jp.xlsx'
result.to_excel(path)

result

In [ ]:
filt_Other = df.region == "Other"
result = df[filt_Other].sort_values('sales', ascending=False).head()[['Name','Genre','Platform','Year','sales']]

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/pop_game_05_sales_other.xlsx'
result.to_excel(path)

result

# 미디어패널 조사

In [ ]:
for i in range(10,22):
  mp_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/media_panel/h{}v28_KMP_csv.csv'.format(i)
  mp_raw = pd.read_csv(mp_path, encoding='cp949')
  print(i, ' : ', mp_raw.shape)
  globals()['mp_h{}_raw'.format(i)] = mp_raw

In [ ]:
for i in globals():
  if i[0:4] == 'mp_h' and i[-4:] == '_raw':
    filt = globals()[i].columns.str.contains('a09001')
    print(i, ' 카운트 : ', globals()[i].iloc[:,np.where(filt)[0]].value_counts())
    print(i, ' 결측치 : ', globals()[i].iloc[:,np.where(filt)[0]].isna().sum())

In [ ]:
mp_h12_raw.h12a09001.unique()

In [ ]:
filt = mp_h12_raw.h12a09001 == ' '
mp_h12_raw.drop(index=mp_h12_raw[filt].index, inplace=True)
mp_h12_raw.h12a09001.unique()

In [ ]:
mp_h12_raw.h12a09001 = mp_h12_raw.h12a09001.astype(int)

In [ ]:
for i in globals():
  if i[0:4] == 'mp_h' and i[-4:] == '_raw':
    filt = globals()[i].columns.str.contains('a09001')
    print(i, ' 카운트 : ', globals()[i].iloc[:,np.where(filt)[0]].value_counts())
    print(i, ' 결측치 : ', globals()[i].iloc[:,np.where(filt)[0]].isna().sum())

In [ ]:
for i in globals():
  if i[0:4] == 'mp_h' and i[-4:] == '_raw':
    filt = globals()[i].columns.str.contains('a09001')
    print(i, globals()[i].iloc[:,np.where(filt)[0]].value_counts(normalize=True)[1])

In [ ]:
merge_mp = mp_h10_raw.hid

merge_mp = pd.merge(left = merge_mp, right = mp_h10_raw[['hid','h10a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h11_raw[['hid','h11a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h12_raw[['hid','h12a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h13_raw[['hid','h13a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h14_raw[['hid','h14a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h15_raw[['hid','h15a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h16_raw[['hid','h16a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h17_raw[['hid','h17a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h18_raw[['hid','h18a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h19_raw[['hid','h19a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h20_raw[['hid','h20a09001']], how = 'outer', on = 'hid')
merge_mp = pd.merge(left = merge_mp, right = mp_h21_raw[['hid','h21a09001']], how = 'outer', on = 'hid')

merge_mp

In [ ]:
merge_mp.columns

In [ ]:
merge_mp.rename(columns={
                        'h10a09001' : '2010',
                        'h11a09001' : '2011',
                        'h12a09001' : '2012',
                        'h13a09001' : '2013',
                        'h14a09001' : '2014',
                        'h15a09001' : '2015',
                        'h16a09001' : '2016',
                        'h17a09001' : '2017',
                        'h18a09001' : '2018',
                        'h19a09001' : '2019',
                        'h20a09001' : '2020',
                        'h21a09001' : '2021'
                        }, inplace=True)
merge_mp.columns

In [ ]:
merge_mp.columns[1:]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

consol_rate = []

for year in merge_mp.columns[1:]:
  consol_rate.append(merge_mp[year].value_counts(normalize=True)[1]*100)

ax.plot(merge_mp.columns[1:], consol_rate, lw=3)


## 그래프 설정
plt.title('Rate Consol in Home', fontsize=22, fontweight = 'bold', pad=15)


plt.xlabel('Year', fontsize=14)
plt.ylabel('Rate of Consol', fontsize=14)

plt.show()

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/mediapanel_01_rate_consol.png'
fig.savefig(path, dpi=150, facecolor='#eeeeee')

In [ ]:
for i in range(17,22):
  mp_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/media_panel/p{}v28_KMP_csv.csv'.format(i)
  mp_raw = pd.read_csv(mp_path, encoding='cp949')
  print(i, ' : ', mp_raw.shape)
  globals()['mp_p{}_raw'.format(i)] = mp_raw

In [ ]:
for i in globals():
  if i[0:4] == 'mp_p' and i[-4:] == '_raw':
    filt = globals()[i].columns.str.contains('c05043')
    print(i, ' 카운트 : ', globals()[i].iloc[:,np.where(filt)[0]].value_counts())
    print(i, ' 결측치 : ', globals()[i].iloc[:,np.where(filt)[0]].isna().sum())

In [ ]:
mp_p = mp_p17_raw.pid

mp_p = pd.merge(left = mp_p, right = mp_p17_raw[['pid','p17c05043']], how = 'outer', on = 'pid')
mp_p = pd.merge(left = mp_p, right = mp_p18_raw[['pid','p18c05043']], how = 'outer', on = 'pid')
mp_p = pd.merge(left = mp_p, right = mp_p19_raw[['pid','p19c05043']], how = 'outer', on = 'pid')
mp_p = pd.merge(left = mp_p, right = mp_p20_raw[['pid','p20c05043']], how = 'outer', on = 'pid')
mp_p = pd.merge(left = mp_p, right = mp_p21_raw[['pid','p21c05043']], how = 'outer', on = 'pid')

mp_p

In [ ]:
mp_p.rename(columns={
                    'p17c05043' : '2017',
                    'p18c05043' : '2018',
                    'p19c05043' : '2019',
                    'p20c05043' : '2020',
                    'p21c05043' : '2021'
                    }, inplace=True)
mp_p.columns

In [ ]:
mp_p['2017'].value_counts(normalize=True).sort_index()['1']

In [ ]:
mp_p['2017'].value_counts(normalize=True).sort_index()

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

smart_phone = []
tablet = []
PC_laptop = []
consol = []

for year in mp_p.columns[1:]:
  smart_phone.append(mp_p[year].value_counts(normalize=True).sort_index()['1']*100)
  tablet.append(mp_p[year].value_counts(normalize=True).sort_index()['2']*100)
  PC_laptop.append(mp_p[year].value_counts(normalize=True).sort_index()['3']*100)
  consol.append(mp_p[year].value_counts(normalize=True).sort_index()['4']*100)

ax1.plot(mp_p.columns[1:], smart_phone, label='Smart Phone')
ax1.plot(mp_p.columns[1:], tablet, label='Tablet')
ax1.plot(mp_p.columns[1:], PC_laptop, label='PC & Laptop')
ax1.plot(mp_p.columns[1:], consol, label='Consol')

## 그래프 설정
plt.title('Game Kit', fontsize=22, fontweight = 'bold', pad=15)

plt.xlabel('Year', fontsize=14)
plt.ylabel('Rate', fontsize=14)

ax1.legend()
plt.show()

path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/results/mediapanel_02_game_kit.png'
fig.savefig(path, dpi=150, facecolor='#eeeeee')